In [ ]:
!pip install pulp
import pulp as lp
import random
import math
import statistics  # Import statistics module at the top


# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 1.0  # Weight for PGC
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]

# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_fixed = lp.LpProblem("Fixed_Assignment_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_fixed = lp.LpVariable.dicts("x_fixed", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                          for k in timeslots for day in days), 0, 1, lp.LpBinary)
y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
                                            for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for any timeslot in a week
y_shared_new = lp.LpVariable.dicts("y_shared_new",
                                   ((d, t) for d in districts for t in range(total_trucks)), 0, 1, lp.LpBinary)
g_fixed = lp.LpVariable.dicts("g_fixed", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_fixed[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_fixed += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)

    # Constraint to express absolute deviation from mean
    model_fixed += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_fixed += alpha * TCT(x_fixed) - beta * TGC(g_fixed) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_fixed += g_fixed[(d, day)] == lp.lpSum(
            x_fixed[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_Fixed_{d}_{day}"
    # Frequency requirement constraints
   # model_fixed += lp.lpSum(
   #     x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
   # model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
     #                       days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            model_fixed += schedule_used[(d, day, k)] <= lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
            model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[(d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
model_fixed += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
    x_fixed[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
    days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)
#for t in range(total_trucks):
  #  model_fixed += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"

# Additional binary variable to indicate if a truck is assigned to a district at all
z = lp.LpVariable.dicts("z", ((t, d) for t in range(total_trucks) for d in districts), 0, 1, lp.LpBinary)

# Link x_fixed and z variables
for t in range(total_trucks):
    for d in districts:
        # Link x_fixed and z: if z[t, d] is 1, truck t can be assigned to district d on specific (day, timeslot) combinations
        model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for day in days for k in timeslots) <= z[(t, d)] * len(days) * len(timeslots)

# Ensure each truck is assigned to exactly one district across all days and timeslots
for t in range(total_trucks):
    model_fixed += lp.lpSum(z[(t, d)] for d in districts) == 1, f"OneDistrictP#_{t}"

     #   model_fixed += (lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) > 0)
# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
#for d in districts:
  #  for t in range(total_trucks):
  #      model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
                                                                                      #          3], f"Truck_Frequency_{d}_{t}"

# Solve the model
model_fixed.solve()

# Display results
print("Fixed Assignment Model:")
print("Status:", lp.LpStatus[model_fixed.status])
print("Objective Value:", lp.value(model_fixed.objective))

# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_fixed[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_fixed[(d, t, k, day)])}")
print(lp.value(TCT(x_fixed)))
print(lp.value(TGC(g_fixed)))
print(lp.value(proportions))

